# Integrate well logs divided in different files

With this code you will be able to integrate DT and RHOB logs in one .las file.

In [ ]:
import las
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.gridspec as gridspec

import math

from IPython.display import Image

import pandas as pd

import sys

In [ ]:
las_files = ('perfis/1RJS_0051__RJ_1RJS_0051__RJ_BHC_00003.las', 'perfis/1RJS_0051__RJ_1RJS_0051__RJ_FDC_CNL_00001.las', 'perfis/1RJS_0051__RJ_1RJS_0051__RJ_FDC_CNL_GR_00004.las', 'perfis/1RJS_0051__RJ_1RJS_0051__RJ_IES_00005.las', 'perfis/1RJS_0051__RJ_1RJS_0051__RJ_ISF_BHC_GR_00002.las', 'perfis/1RJS_0051__RJ_1RJS_0051__RJ_SIGEO__013_00006.las')

In [ ]:
N_files = len(las_files)
N_files

In [ ]:
for i in range(N_files):
    globals()['log%s' % i] = las.LASReader(las_files[i])

In [ ]:
log0.step

In [ ]:
log2.data

In [ ]:
np.set_printoptions(precision=4)

## Reading and assigning the logs:

**Important!** This code needs to be customized for each well. Unfortunately I still didn't find a way to integrate all of them with a general code. 

In [ ]:
START = []
END = []
DEP = []
DEPT = []

N_files = len(las_files)

for i in range(N_files):              # Creating an empty array indexed with the depth
    
    globals()['log_%s' % i] = las.LASReader(las_files[i])
    np.set_printoptions(precision=4)
    
    globals()['DEPT_%s' % i] = globals()['log_%s' % i].data['DEPT']
    
    START_logs =  globals()['DEPT_%s' % i][0]
    START = np.append(START, START_logs)
    
    END_logs =  globals()['DEPT_%s' % i][-1]
    END = np.append(END, END_logs)
    
    DEPT_start = np.argmin(START)
    DEPT_end = np.argmax(END)
    
    sample_int = globals()['log_%s' % i].well.STEP.value
    N_samples = int((END[DEPT_end] - START[DEPT_start])/sample_int)+1
    
    Ini = START[DEPT_start] 
    
DEPT = np.append(DEPT, Ini)
    
for i in range(N_samples):   
    
    DE = Ini + sample_int
    DEP = np.append(DEP, DE)
    Ini = DE     
    
DEPT = np.append(DEPT, DEP)



counter = 0

count_d = []
count_r = []

for i in range(N_files):                  # counting the number of columns the same log appears in each file
        
    with open(las_files[i]) as f:
        contents = f.read()
        count = contents.count("Sonic")
        count_d = np.append(count_d, count)
        
    with open(las_files[i]) as f:
        contents = f.read()
        count = contents.count("Bulk")
        count_r = np.append(count_r, count)
          
            
            
# Analysis of the DT log


D = np.empty(N_samples+1,)          # Creating and empty array to receive the values
D[:] = np.nan
DT = D

count_d_i = np.where(count_d>0)

for i in range(np.asarray(count_d_i).size):
        
            
    if count_d[count_d_i[0][i]] == 1:                          # verifies if the same log is divided in different files
        
        
        globals()['log_S_%s' % i] = las.LASReader(las_files[count_d_i[0][i]])  # saves the logs which has DT log
        
        globals()['DEPT_S_%s' % i] = globals()['log_S_%s' % i].data['DEPT']   # Stores the depths of each file in different parameters
               
        
        x = 'DT_'
        y = i
        z = count_d_i[0][i]
            
        globals()['%s%s%s' % (x,y,z)] = globals()['log_S_%s' % i].data['DT']
        
        xv = 'valid_S_'
        yv = i
        zv = count_d_i[0][i]
        
        globals()['%s%s%s' % (xv,yv,zv)] = np.where(globals()['%s%s%s' % (x,y,z)] > 0)   # finding the set of valid values 

        globals()['idx_v_S_%s' % i] = globals()['%s%s%s' % (xv,yv,zv)][0][0]

        globals()['START_S_%s' % i] =  globals()['DEPT_S_%s' % i][globals()['idx_v_S_%s' % i]]  # finding the depth of the first valid value
        
        globals()['idx_S_start_%s' % i] = np.abs(globals()['START_S_%s' % i] - DEPT).argmin()   # index where the DT log starts in DEPT
        
        globals()['idx_v_E_%s' % i] = globals()['%s%s%s' % (xv,yv,zv)][0][-1] # finding the index of the last valid value 
        
        globals()['END_S_%s' % i] =  globals()['DEPT_S_%s' % i][globals()['idx_v_E_%s' % i]]   # finding the depth of the last valid value
        
        globals()['idx_S_end_%s' % i]  = np.abs(globals()['END_S_%s' % i] - DEPT).argmin()   # index where the DT log ends in DEPT

        
        DT[(globals()['idx_S_start_%s' % i]):(globals()['idx_S_end_%s' % i])+1] = globals()['%s%s%s' % (x,y,z)][globals()['%s%s%s' % (xv,yv,zv)][0][0]:globals()['%s%s%s' % (xv,yv,zv)][0][-1]+1]
        
        
    if count_d[count_d_i[0][i]] > 1:   # verifies if the same log is divided in different columns
        
        globals()['log_S_%s' % i] = las.LASReader(las_files[count_d_i[0][i]])  # saves the logs which has DT log
        
        globals()['DEPT_S_%s' % i] = globals()['log_S_%s' % i].data['DEPT']   # Stores the depths of each file in different parameters
               
        
        x = 'DT_'
        y = i
        z = count_d_i[0][i]
                
        globals()['%s%s%s' % (x,y,z)] = globals()['log_S_%s' % i].data['DT']  # reading the first column
                
        xv = 'valid_S_'
        yv = i
        zv = count_d_i[0][i]
        
        globals()['%s%s%s' % (xv,yv,zv)] = np.where(globals()['%s%s%s' % (x,y,z)] > 0)
          
        globals()['idx_v_S_%s' % i] = globals()['%s%s%s' % (xv,yv,zv)][0][0]      # finding the index of the first valid value

        globals()['START_S_%s' % i] =  globals()['DEPT_S_%s' % (i)][globals()['idx_v_S_%s' % i]]  # finding the depth of the first valid value
        
        globals()['idx_S_start_%s' % i] = np.abs(globals()['START_S_%s' % i] - DEPT).argmin()   # index where the RHOB log starts in DEPT

        globals()['idx_v_E_%s' % i] = globals()['%s%s%s' % (xv,yv,zv)][0][-1] # finding the index of the last valid value
          
        globals()['END_S_%s' % i] =  globals()['DEPT_S_%s' % i][globals()['idx_v_E_%s' % i]]   # finding the depth of the last valid value
        
        globals()['idx_S_end_%s' % i]  = np.abs(globals()['END_S_%s' % i] - DEPT).argmin()   # index where the DT log ends in DEPT
                  
        DT[(globals()['idx_S_start_%s' % i]):(globals()['idx_S_end_%s' % i])+1] = globals()['%s%s%s' % (x,y,z)][globals()['%s%s%s' % (xv,yv,zv)][0][0]:globals()['%s%s%s' % (xv,yv,zv)][0][-1]+1]
     
    
   
        
        for i in range(int(count_d[count_d_i[0][i]]) -1):  # Stores the logs divided in different columns in different parameters
                
            x = 'DT_'
            y = i
            z = count_d_i[0][i]
        
            globals()['DT_%s' % (i+1)] = globals()['log_S_%s' % i].data['DT_%s' % (i+1)]
            
                
            globals()['valid_s_%s' % (i+1)] = np.where(globals()['DT_%s' % (i+1)] > 0)
                    
            globals()['%s%s%s' % (x,y,z)][globals()['valid_s_%s' % (i+1)][:]] = globals()['DT_%s' % (i+1)][globals()['valid_s_%s' % (i+1)]]    

            globals()['idx_v_S_%s' % i] = globals()['valid_s_%s' % (i+1)][0][0]      # finding the index of the first valid value
            
            globals()['START_S_%s' % i] =  globals()['DEPT_S_%s' % (i)][globals()['idx_v_S_%s' % i]]  # finding the depth of the first valid value
                      
            globals()['idx_S_start_%s' % i] = np.abs(globals()['START_S_%s' % i] - DEPT).argmin()   # index where the RHOB log starts in DEPT
            
            globals()['idx_v_E_%s' % i] = globals()['valid_s_%s' % (i+1)][0][-1] # finding the index of the last valid value

            globals()['END_S_%s' % i] =  globals()['DEPT_S_%s' % i][globals()['idx_v_E_%s' % i]]   # finding the depth of the last valid value
            
            globals()['idx_S_end_%s' % i]  = np.abs(globals()['END_S_%s' % i] - DEPT).argmin()   # index where the DT log ends in DEPT
        
                     
            DT[(globals()['idx_S_start_%s' % i]):(globals()['idx_S_end_%s' % i])+1] = globals()['%s%s%s' % (x,y,z)][globals()['valid_s_%s' % (i+1)][0][0]:globals()['valid_s_%s' % (i+1)][0][-1]+1]
               

null = np.where(np.nan_to_num(DT)<0)
DT[null] = np.nan




# Analysis of the RHOB log



R = np.empty(N_samples+1,)           # Creating an empty array to receive the values
R[:] = np.nan
RHOB = R

count_r_i = np.where(count_r>0) 

for i in range(np.asarray(count_r_i).size):
        
            
    if count_r[count_r_i[0][i]] == 1:                          # verifies if the same log is divided in different files
        
        
        globals()['log_B_%s' % i] = las.LASReader(las_files[count_r_i[0][i]])  # saves the logs which has DT log
        
        globals()['DEPT_B_%s' % i] = globals()['log_B_%s' % i].data['DEPT']   # Stores the depths of each file in different parameters
               
        
        x = 'RHOB_'
        y = i
        z = count_r_i[0][i]
            
        globals()['%s%s%s' % (x,y,z)] = globals()['log_B_%s' % i].data['RHOB']
        
        xv = 'valid_R_'
        yv = i
        zv = count_r_i[0][i]
        
        globals()['%s%s%s' % (xv,yv,zv)] = np.where(globals()['%s%s%s' % (x,y,z)] > 0)   # finding the set of valid values 

        globals()['idx_v_B_%s' % i] = globals()['%s%s%s' % (xv,yv,zv)][0][0]

        globals()['START_B_%s' % i] =  globals()['DEPT_B_%s' % i][globals()['idx_v_B_%s' % i]]  # finding the depth of the first valid value
        
        globals()['idx_B_start_%s' % i] = np.abs(globals()['START_B_%s' % i] - DEPT).argmin()   # index where the DT log starts in DEPT
        
        globals()['idx_v_E_%s' % i] = globals()['%s%s%s' % (xv,yv,zv)][0][-1] # finding the index of the last valid value 
        
        globals()['END_B_%s' % i] =  globals()['DEPT_B_%s' % i][globals()['idx_v_E_%s' % i]]   # finding the depth of the last valid value
        
        globals()['idx_B_end_%s' % i]  = np.abs(globals()['END_B_%s' % i] - DEPT).argmin()   # index where the DT log ends in DEPT

        
        RHOB[(globals()['idx_B_start_%s' % i]):(globals()['idx_B_end_%s' % i])+1] = globals()['%s%s%s' % (x,y,z)][globals()['%s%s%s' % (xv,yv,zv)][0][0]:globals()['%s%s%s' % (xv,yv,zv)][0][-1]+1]
        
        
    if count_r[count_r_i[0][i]] > 1:   # verifies if the same log is divided in different columns
        
        globals()['log_B_%s' % i] = las.LASReader(las_files[count_r_i[0][i]])  # saves the logs which has DT log
        
        globals()['DEPT_B_%s' % i] = globals()['log_B_%s' % i].data['DEPT']   # Stores the depths of each file in different parameters
               
        
        x = 'RHOB_'
        y = i
        z = count_r_i[0][i]
                
        globals()['%s%s%s' % (x,y,z)] = globals()['log_B_%s' % i].data['RHOB']  # reading the first column
                
        xv = 'valid_B_'
        yv = i
        zv = count_r_i[0][i]
        
        globals()['%s%s%s' % (xv,yv,zv)] = np.where(globals()['%s%s%s' % (x,y,z)] > 0)
          
        globals()['idx_v_B_%s' % i] = globals()['%s%s%s' % (xv,yv,zv)][0][0]      # finding the index of the first valid value

        globals()['START_B_%s' % i] =  globals()['DEPT_B_%s' % (i)][globals()['idx_v_B_%s' % i]]  # finding the depth of the first valid value
        
        globals()['idx_B_start_%s' % i] = np.abs(globals()['START_B_%s' % i] - DEPT).argmin()   # index where the RHOB log starts in DEPT

        globals()['idx_v_E_%s' % i] = globals()['%s%s%s' % (xv,yv,zv)][0][-1] # finding the index of the last valid value
          
        globals()['END_B_%s' % i] =  globals()['DEPT_B_%s' % i][globals()['idx_v_E_%s' % i]]   # finding the depth of the last valid value
        
        globals()['idx_B_end_%s' % i]  = np.abs(globals()['END_B_%s' % i] - DEPT).argmin()   # index where the DT log ends in DEPT
                  
        
        RHOB[(globals()['idx_B_start_%s' % i]):(globals()['idx_B_end_%s' % i])+1] = globals()['%s%s%s' % (x,y,z)][globals()['%s%s%s' % (xv,yv,zv)][0][0]:globals()['%s%s%s' % (xv,yv,zv)][0][-1]+1]
     
    
   
        
        for i in range(int(count_r[count_r_i[0][i]]) -1):  # Stores the logs divided in different columns in different parameters
                
            x = 'RHOB_'
            y = i
            z = count_r_i[0][i]
        
            globals()['RHOB_%s' % (i+1)] = globals()['log_B_%s' % i].data['RHOB_%s' % (i+1)]
            
                
            globals()['valid_b_%s' % (i+1)] = np.where(globals()['RHOB_%s' % (i+1)] > 0)
                    
            globals()['%s%s%s' % (x,y,z)][globals()['valid_b_%s' % (i+1)][:]] = globals()['RHOB_%s' % (i+1)][globals()['valid_b_%s' % (i+1)]]    

            globals()['idx_v_B_%s' % i] = globals()['valid_b_%s' % (i+1)][0][0]      # finding the index of the first valid value
            
            globals()['START_B_%s' % i] =  globals()['DEPT_B_%s' % (i)][globals()['idx_v_B_%s' % i]]  # finding the depth of the first valid value
                      
            globals()['idx_B_start_%s' % i] = np.abs(globals()['START_B_%s' % i] - DEPT).argmin()   # index where the RHOB log starts in DEPT
            
            globals()['idx_v_E_%s' % i] = globals()['valid_b_%s' % (i+1)][0][-1] # finding the index of the last valid value

            globals()['END_B_%s' % i] =  globals()['DEPT_B_%s' % i][globals()['idx_v_E_%s' % i]]   # finding the depth of the last valid value
            
            globals()['idx_B_end_%s' % i]  = np.abs(globals()['END_B_%s' % i] - DEPT).argmin()   # index where the DT log ends in DEPT
        
                     
            RHOB[(globals()['idx_B_start_%s' % i]):(globals()['idx_B_end_%s' % i])+1] = globals()['%s%s%s' % (x,y,z)][globals()['valid_b_%s' % (i+1)][0][0]:globals()['valid_b_%s' % (i+1)][0][-1]+1]
               



null = np.where(np.nan_to_num(RHOB)<0)
RHOB[null] = np.nan



In [ ]:
DT

In [ ]:
RHOB

In [ ]:
DEPT

In [ ]:
DEPT.size

## Reading the start and end of the depth:

In [ ]:
DEPT_start = DEPT[0]
DEPT_end = DEPT[-1]

In [ ]:
print DEPT_start

In [ ]:
print DEPT_end

## Exporting integrated logs to a txt file

In [ ]:
Depth = np.array([DEPT])
Depth.T

In [ ]:
DT = np.array([DT])
DT.T

In [ ]:
RHOB = np.array([RHOB])
RHOB.T

In [ ]:
logs_DT_RHOB_int = np.concatenate((Depth.T, DT.T, RHOB.T), axis=1)
logs_DT_RHOB_int

In [ ]:
np.savetxt('logs_DT_RHOB_int.txt', (logs_DT_RHOB_int), fmt='%8.4f', delimiter='    ', newline='\n', header='', footer='', comments='# ') 

## Creating a .las file manually

Now, to import the data into an integration software you need to create a new .las file with the header and the integrated data. Pay attention to change at the header the depth related to the start and end of the well and replace the null value by 'nan'. 

To see the header of well and create a new file .las, use the following command:

In [21]:
import sys
orig_stdout = sys.stdout
sys.stdout=open('out.las','w')

In [22]:
import numpy as np
with open('perfis/1RJS_0051__RJ_1RJS_0051__RJ_BHC_00003.las', 'r') as f:
    for line in f:
        if line.lstrip().startswith('~A'):
            break
        f = np.append(f, (line.rstrip()))
        print(line.rstrip())

In [23]:
# Insert the column RHOB to the file

In [24]:
sys.stdout=open('out.las','a')
print ('RHOB.                                                    :     3  Bulk density') 

In [25]:
# Insert the ASCII section

In [26]:
sys.stdout=open('out.las','a')
print(line.rstrip())

In [27]:
with open('logs_DT_RHOB_int.txt', 'r') as data:
    for line in data:
        if line.lstrip().startswith('~A'):
            break
        data = np.append(data, (line.rstrip()))
        print(line.rstrip())
data = (data, (line.rstrip()))